In [ ]:
import sys, os, numpy as np, pandas as pd, joblib
from sklearn.model_selection import train_test_split
sys.path.append(os.path.abspath(os.path.join('..')))
from src.classification_library import load_data, clean_data

In [ ]:
# tags=["parameters"]
CLEANED_PATH = '../data/processed/cleaned.parquet'
OUTPUT_SEMI_DATASET_PATH = '../data/processed/dataset_for_semi.parquet'
LABEL_MISSING_FRACTION = 0.95

In [ ]:
df = pd.read_parquet(CLEANED_PATH)
df = df.dropna(subset=['PM2.5'])
def create_aqi(x): return 0 if x<=35 else (1 if x<=75 else 2)
df['AQI_Label'] = df['PM2.5'].apply(create_aqi)
feats = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
X = df[feats]
y = df['AQI_Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
y_semi = y_train.copy().values
n_unlabel = int(len(y_train) * float(LABEL_MISSING_FRACTION))
y_semi[:n_unlabel] = -1
joblib.dump({'X_train':X_train, 'y_train_semi':pd.Series(y_semi, index=X_train.index), 'y_test':y_test, 'X_test':X_test}, OUTPUT_SEMI_DATASET_PATH)
print("Done Semi Prep.")